### Обработаем значения

In [1]:
from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower()                             # Конвертировать в нижний регистр
    all_words = re.findall("[a-z0-9']+", text)      # извлечь слова и
    return set(all_words)                           # удалить повторы

assert tokenize("Data Science is science") == {"data", "science", "is"}


### Определим тип для тренировочных данных

In [2]:
from typing import NamedTuple

class Message(NamedTuple):
    text: str
    is_spam: bool

### Классификатор

In [5]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k # Сглаживающий фактор

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Увеличить количество сообщений
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Увеличить количество появлений слов
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
        """Возвращает P(лексема | спам) и P(лексема | неспам)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Перебрать все слова в лексиконе
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            # Если *лексема* появляется в сообщении,
            # то добавить лог вероятность ее встретить
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # В противном случае добавить лог вероятность
            # не встретить ее, т. е. log(1 - вероятность встретить)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

### Тестирование модели

#### тесты

In [6]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

#### Предсказания 

In [10]:
text = "hello spam"

probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam" (присутствует)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham" (не присутствует)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (не присутствует)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (присутствует)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam" (присутствует)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham" (не присутствует)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (не присутствует)
    (1 + 0.5) / (2 + 2 * 0.5)       # "hello" (присутствует)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

## Применение модели

### Загрузка датасета 

In [11]:
from io import BytesIO
import requests
import tarfile

BASE_URL = "https://spamassassin.apache.org/old/publiccorpus"
FILES = ["20021010_easy_ham.tar.bz2",
         "20021010_hard_ham.tar.bz2",
         "20021010_spam.tar.bz2"]

OUTPUT_DIR = 'spam_data'

for filename in FILES:
    content = requests.get(f"{BASE_URL}/{filename}").content

    # Обернуть байты в памяти, чтобы использовать их как "файл"
    fin = BytesIO(content)

    with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
        tf.extractall(OUTPUT_DIR)

C:\Users\ura_s\AppData\Local\Temp\ipykernel_16564\126572364.py:19: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tf.extractall(OUTPUT_DIR)


### Обработка писем

In [13]:
import glob, re


path = 'spam_data/*/*'

data: List[Message] = []
# glob.glob возвращает каждое имя файла,
# которое соответствет поисковому шаблону пути
for filename in glob.glob(path):
    is_spam = "ham" not in filename

    # В письмах имеются несколько мусорных символов;
    # параметр errors='ignore' пропускает их вместо вызова исключения
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break


### Разбиваем данные на train и test

In [22]:
import random
from theory.machine_learning import split_data

random.seed(0) 
train_messages, test_messages = split_data(data, 0.75)

model = NaiveBayesClassifier()
model.train(train_messages)

### Проверка работы модели

In [ ]:
from collections import Counter

predictions = [(message, model.predict(message.text))
               for message in test_messages]

# Будем считать, что спамна вероятность spam_probability > 0.5
# соответствует предсказанию спама, и подсчитаем комбинации
# (фактический спам is_spam, предсказанный спам is_spam)
confusion_matrix = Counter((message.is_spam, spam_probability > 0.5)
                           for message, spam_probability in predictions)

print(confusion_matrix)

precision = confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1])
recall = confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[1,0])
print("precision is ", precision)
print("recall is ", recall)

Counter({(False, False): 670, (True, True): 86, (True, False): 40, (False, True): 29})
precision is  0.7478260869565218
recall is  0.6825396825396826


### Небольшая статистика

In [34]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:

    prob_if_spam, prob_if_ham = model._probabilities(token)

    return prob_if_spam / (prob_if_spam + prob_if_ham)

words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))

print("наиболее спамные слова", words[-10:])
print("наименее спамные слова", words[:10])

наиболее спамные слова ['assistance', '95', 'attn', 'per', 'money', 'clearance', 'sale', 'rates', 'systemworks', 'adv']
наименее спамные слова ['spambayes', 'users', 'razor', 'zzzzteana', 'sadev', 'apt', 'perl', 'ouch', 'spamassassin', 'bliss']
